In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from sklearn.cluster import KMeans


import random 
from random import seed

import time


import numpy as np
import matplotlib.pyplot as plt


# Load features and labels
features_saved = np.loadtxt('Models/Original_model/ll_features_20.txt')
labels_features_saved = np.loadtxt('Models/Original_model/ll_labels_features_20.txt').astype(int)


# Load model - CFR
model = keras.models.load_model('Models/Original_model/mnist_cnn.h5') # Frozen model 

# model_frozen = keras.models.Sequential(model.layers[:-1])  # extract the last layer from the original model
# model_frozen.compile()

model_frozen = keras.models.load_model('Models/Frozen_model/omv_mnist_cnn.h5')

# Print
# model.summary()
# model_frozen.summary()

# Absolute path is needed to load libraries 
ROOT_PATH = os.path.abspath('')
sys.path.append(ROOT_PATH + '/lib')



# Test import
# from lib import simulation_lib
# from lib.simulation_lib import

from lib import Kmeans_lib
from lib.Kmeans_lib import *
# from lib.EvalMetrics import *

(data_train, label_train),(data_test, label_test) = mnist.load_data() # Load data

train_samples = label_train.shape[0]
test_samples  = label_test.shape[0]
img_rows, img_cols = 28, 28

trainLow_samples = np.sum(np.where(label_train < 6, 1, 0))
testLow_samples  = np.sum(np.where(label_test  < 6, 1, 0))

# separate in containers data that is lower and higer than 6
# TRAIN - LOW
data_low_train   = np.zeros([trainLow_samples,28,28])
label_low_train  = np.zeros(trainLow_samples)
#       - HIGH
data_high_train  = np.zeros([train_samples-trainLow_samples,28,28])
label_high_train = np.zeros(train_samples-trainLow_samples)

# TEST - LOW
data_low_test   = np.zeros([testLow_samples,28,28])
label_low_test  = np.zeros(testLow_samples)

#      - HIGH
data_high_test  = np.zeros([test_samples-testLow_samples,28,28])
label_high_test = np.zeros(test_samples-testLow_samples)

j,k = 0,0
for i in range(0,train_samples):  
    if(label_train[i]<6):
        data_low_train[j,:,:] = data_train[i,:,:]
        label_low_train[j]    = label_train[i]
        j+=1
    else:
        data_high_train[k,:,:] = data_train[i,:,:]
        label_high_train[k]    = label_train[i]
        k+=1


j,k = 0,0
for i in range(0,test_samples):
    if(label_test[i]>5):
        data_high_test[k,:,:] = data_test[i,:,:]
        label_high_test[k]    = label_test[i]
        k+=1  
    else:
        data_low_test[j,:,:] = data_test[i,:,:]
        label_low_test[j]    = label_test[i]
        j+=1

# Reshape arrays
data_low_train  = data_low_train.reshape(data_low_train.shape[0], img_rows, img_cols, 1)
data_high_train = data_high_train.reshape(data_high_train.shape[0], img_rows, img_cols, 1)
data_low_test   = data_low_test.reshape(data_low_test.shape[0], img_rows, img_cols, 1)
data_high_test  = data_high_test.reshape(data_high_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

# Normalize the colors from 0-255 to 0-1
data_low_train  = data_low_train.astype(np.float32) / 255.0
data_high_train = data_high_train.astype(np.float32) / 255.0
data_low_test   = data_low_test.astype(np.float32) / 255.0
data_high_test  = data_high_test.astype(np.float32) / 255.0

2022-10-06 14:17:41.589751: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-06 14:17:47.963564: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [124]:
'''Function to compute kmean clustering on the new dataset and the saved features'''
def k_mean_clustering(features_run, features_saved, labels_features_run, labels_features_saved, n_cluster, batch_size):

  # Define initial set of features
  # CURRENTLY SUPPORTS ONLY SEQUENTIAL SELECTION i.e. 0,1,2,3,4,5
  
  
  # labels_init_list = list(range(0, n_cluster))

  labels_init_list = list([1, 9, 5, 0])
  labels_init_list = list([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
  n_cluster = len(labels_init_list)

  # labels_init_list = list([1, 9, 5, 0])
  # labels_init_list = list([1, 2, 3, 4, 5, 6, 7, 8, 9, 0])
  # n_clust_init = len(labels_init_list)

  # Extract from the saved features the labels that we need
  features_saved_init = []
  labels_features_saved_init = []
  # Extract features of digits considered in labels_init_list
  for i in range(0, len(features_saved)):
      if labels_features_saved[i] in labels_init_list:
        features_saved_init.append(features_saved[i,:])
        labels_features_saved_init.append(labels_features_saved[i])
  
  # Convert list to nparray
  features = np.array(features_saved_init)
  labels_features = np.array(labels_features_saved_init)  

  # Concateno al vettore delle features iniziali le features della nuova batch da analizzare
  # features = np.concatenate((features, features_run))
  # labels_features = np.append(labels_features, labels_features_run).astype(int)

  # KMean Clustering
  k_mean = create_k_mean(features, n_cluster)

  # Find pseudolabels for each new image
  # Pseudolabels are computed by looking at the confusion matrix of the saved dataset (where ground truth is known)
  clusters_features_saved = list(k_mean.labels_[0:len(labels_features_saved_init)])
  labels_features_saved_init = list(labels_features_saved_init)
  cluster_list = list(range(0,n_cluster))
  map_clu2lbl, map_lbl2clu = cluster_to_label(clusters_features_saved, labels_features_saved_init, cluster_list, labels_init_list)

  # print(map_clu2lbl)

  print(set(labels_features))
  print(set(labels_features_saved_init))

  clusters_features = k_mean.labels_

  # Find pseudo label (labels obtained from the model of each image
  pseudolabels_features = np.zeros(len(clusters_features), dtype=int)
  
  for i in range(0, len(clusters_features)):
    pseudolabels_features[i] = map_clu2lbl[clusters_features[i]]
  

  print(clusters_features)
  print(pseudolabels_features)
  print(labels_features)

  print(len(clusters_features))

  # labels_features = map_clu2lbl[]

  # conf_matrix(clusters_features_saved, labels_features_saved_init, , labels_init_list)
  
  # Evaluation metrics
  return k_mean


  

In [128]:
'''Function to compute kmean clustering on the new dataset and the saved features'''
def k_mean_clustering2(features_run, features_saved, labels_features_run, labels_features_saved, n_cluster, batch_size):
# def k_mean_clustering2(features_saved, labels_features_saved):

  labels_init_list = list([1, 9, 5, 0])
  labels_init_list = list([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
  n_cluster = len(labels_init_list)

  # Extract from the saved features the labels that we need
  features_saved_init = []
  labels_features_saved_init = []
  # Extract features of digits considered in labels_init_list
  for i in range(0, len(features_saved)):
      if labels_features_saved[i] in labels_init_list:
        features_saved_init.append(features_saved[i,:])
        labels_features_saved_init.append(labels_features_saved[i])
  
  # Convert list to nparray
  features = np.array(features_saved_init)
  labels_features = np.array(labels_features_saved_init)  

  kmeans = KMeans(n_cluster)

  # Assegno i centroidi iniziali e calcolo le varianze dei cluster
  # Initialize dictionary used to store images related to each cluster
  # clu_images = dict.fromkeys(range(0, n_clust_init), [])
  # creates dictionary using dictionary comprehension -> list [] is mutable object
  features_saved_dict = { key : [] for key in labels_init_list}

  for i in range(0, len(features_saved_init)):
      lbl = labels_features_saved_init[i]
      features_saved_dict[lbl].append(features_saved_init[i])

  print(features_saved_dict[2][1][230])

  # cluster_mean_dict = { key : [] for key in labels_init_list}
  cluster_mean = []
  # Converto list-of-arrays in 2D array
  for key in labels_init_list:
    features_saved_dict[key] = np.array(features_saved_dict[key])
    cluster_mean.append(np.mean(features_saved_dict[key], axis=0))

  
  # Converto in 2D nparray
  cluster_mean = np.array(cluster_mean)

  # Bisogna fare un fit per evitare errori -> passo una immagine per ogni cluster
  init_dataset = np.zeros([10, 512])

  #init_dataset[0,:] = 

  kmeans.fit(cluster_mean)

  print(kmeans.predict(cluster_mean))

  img1 = features_saved_dict[1][1]


  img1 = cluster_mean[1]


  # img1 = img2

  print("Img1 shape", img1.shape)

  # print(cluster_mean.shape)

  img1 = features_saved_dict[1][1]

  img1 = img1.reshape(1, -1)

  print(kmeans.predict(img1))
  print(kmeans.score(img1))
  #print(kmeans.transform(img1).shape) 

  # kmeans.cluster_centers_ = np.array(cluster_mean)
  # kmeans.predict(features_run)
  #cd = kmeans.cluster_centers_
  #kmeans.cluster_centers_ = 
  # print(cd.shape)


  # Evaluation metrics

  #print(features_saved_dict[1][1].shape)

  #img1 = features_saved_dict[1][1]
  #img1.reshape(1, -1)
  #print(img1.shape)
  #kmeans.predict(img1)


  # FIT NEW DATA

  return 


###########

# k_mean_clustering2(features_saved, labels_features_saved)
if True:
  n = range(0, 10)
  images = data_train[n,:,:]
  labels_features_run = label_train[n]

  features_run = []
  for img in images:
    features_run.append(Custom_Layer(model).ML_frozen.predict(img.reshape((1,28,28,1)), verbose = False)/1000)

  features_run = np.array(features_run).squeeze(axis=1)

  n_cluster = 5 # ARGOMENTO NON USATO
  # AGGIORNARE PASSANDO LA LISTA DI LABELS DA CLUSTERIZZARE
  batch_size = 10

  kmean = k_mean_clustering2(features_run, features_saved, labels_features_run, labels_features_saved, n_cluster, batch_size)



0.0
[3 2 8 4 1 6 5 7 0 9]
Img1 shape (512,)
[2]
-93.53816038717422


In [26]:
from tabnanny import verbose


if True:
  n = range(0, 10)
  images = data_train[n,:,:]
  labels_features_run = label_train[n]

  features_run = []
  for img in images:
    features_run.append(Custom_Layer(model).ML_frozen.predict(img.reshape((1,28,28,1)), verbose = False)/1000)

  features_run = np.array(features_run).squeeze(axis=1)

  n_cluster = 5 # ARGOMENTO NON USATO
  # AGGIORNARE PASSANDO LA LISTA DI LABELS DA CLUSTERIZZARE
  batch_size = 10

  kmean = k_mean_clustering2(features_run, features_saved, labels_features_run, labels_features_saved, n_cluster, batch_size)


20


In [3]:
def cluster_to_label(clusters_features, labels_features, cluster_list, labels_init_list):

  # 1: map labels to rows: e.g. [0, 5, 9, 1] -> [1, 3, 2, 0]
  # VECCHIO CODICE, MIGLIORATO USANDO LE CLUSTER LABELS..
  # cluster_label_idx = []
  # for lbl in labels_features:
  # cluster_label_idx.append(labels_features.index(lbl))

  # 2: compute Confusion matrix (for the saved features)
  cmtx = confusion_matrix(clusters_features, labels_features, cluster_list, labels_init_list)

  # PREVIOUS SCRIPT USING CONFUUSION_MATRIX FROM SKLEARN
  # from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
  # cmtx = confusion_matrix(clusters_features, labels_features)
  # Remove empty rows and cols
  # cmtx = cmtx[~np.all(cmtx == 0, axis=1)]
  # cmtx = cmtx[:,~np.all(cmtx == 0, axis=0)]

  # cmtxdisp = ConfusionMatrixDisplay(cmtx) # , display_labels =labels_init_list 

  # Map true-labels to clusters
  # cmtx = np.array(cmtx)

  print(cmtx)

  print("labels saved", labels_features, len(labels_features))
  print("cllusters saved", clusters_features, len(clusters_features))

  map_idx = np.argmax(cmtx, axis = 1)  

  # Find duplicates and fix..


  print(map_idx)

  # Fill dictionary with map
  map_clu2lbl = {}
  map_lbl2clu = {}
  # labels_init_list_sorted = labels_init_list.sort()
  labels_init_list.sort()
  for i in range(0, len(map_idx)):
    map_clu2lbl[map_idx[i]] = labels_init_list[i]
    map_lbl2clu[labels_init_list[i]] = map_idx[i]
  
  # Mapping dictionary
  # map_clu2lbl -> cluster: label
  # map_lbl2clu -> label: cluster


  print(map_lbl2clu)

  return map_clu2lbl, map_lbl2clu


'''Function to compute confusion matrix between cluster and labels'''
def confusion_matrix(clusters_features_saved, labels_features_saved_init, cluster_list, labels_list):

  cmtx = np.zeros([len(labels_list), len(cluster_list)])

  for i in range(0, len(clusters_features_saved)):

    cluster = clusters_features_saved[i]
    label = labels_features_saved_init[i]

    # Find indices
    m = labels_list.index(label)
    n = cluster_list.index(cluster)
    cmtx[m,n] += 1
  return cmtx